## Mahalakshmi S

## 205229118

# Lab19. Text Classification using Glove model

### Imports

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk , os, zipfile
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding,SimpleRNN,LSTM,Bidirectional,Dropout,Conv1D,MaxPooling1D,Flatten,Activation
from nltk.corpus import stopwords
nltk.download('stopwords')
",".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
from sklearn.model_selection import train_test_split

### pre-processing the text

In [ ]:
gl = pd.read_csv('glove-lab-dataset.csv')
gl.head()

In [ ]:
gl.head()

In [ ]:
gl.tail()

In [ ]:
y =gl['category']
X=[]
for review in gl['text']:
  filtered_sentence = [w.lower() for w in review.split() if not w in STOPWORDS]
  X.append(filtered_sentence)
X = pd.Series(X)  

In [ ]:
label_tokenizer = Tokenizer()

#label_tokenizer_train.fit_on_texts(y_train)
label_tokenizer.fit_on_texts(y)

label_seq = np.array(label_tokenizer.texts_to_sequences(y))

In [ ]:
vocab_size = 5000
embedding_dim = 64
max_length = 400
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
X_token = Tokenizer(num_words=5000,oov_token=oov_tok)
X_token.fit_on_texts(X)
word_index = X_token.word_index
X_sequence = X_token.texts_to_sequences(X)

In [ ]:
dict(list(word_index.items())[0:20])

In [ ]:
X_padded = pad_sequences(X_sequence,maxlen=max_length,padding=padding_type)

In [ ]:
print(label_seq.shape)

In [ ]:
print(X_padded.shape)

### Dataset Preparation:

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_padded,label_seq,train_size=0.7)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_test.shape)
print(y_test.shape)

### LSTM Model Creation

In [ ]:
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim))
model1.add(LSTM(embedding_dim))
model1.add(Dense(embedding_dim, activation='relu'))
model1.add(Dense(6,activation='softmax'))

In [ ]:
model1.summary()

In [ ]:
model1.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
history1=model1.fit(X_train,y_train,verbose=2,validation_data=(X_test,y_test),batch_size=32,epochs=20)

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])

plt.title('model loss')
plt.xlabel('loss')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
score = model1.evaluate(X_test,y_test,verbose=0)
print("loss:  ",score[0])
print("accuarcy:  ",score[1])

### CNN-LSTM Model Creation

In [ ]:
model3 = Sequential()
model3.add(Embedding(vocab_size, embedding_dim))
model3.add(Conv1D(filters=32,kernel_size=3,strides=1,activation='relu'))
model3.add(MaxPooling1D((3)))
model3.add(LSTM(embedding_dim))
model3.add(Dense(128,activation= 'relu'))
model3.add(Dense(6,activation='softmax'))

In [ ]:
model3.summary()

In [ ]:
model3.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
history3 = model3.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,epochs=20,batch_size=32)

In [ ]:
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])

plt.title('model loss')
plt.xlabel('loss')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
score = model3.evaluate(X_test,y_test,verbose=0)
print("loss:  ",score[0])
print("accuracy:  ",score[1])

### Pre-trained Model Creation

In [ ]:
!wget --no-check-certificate \
     http://nlp.stanford.edu/data/glove.6B.zip \
     -O /tmp/glove.6B.zip

In [ ]:
with zipfile.ZipFile('/tmp/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/glove')

In [ ]:
embeddings_index = {}
f = open('/tmp/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
embedding_layer = Embedding(input_dim=len(word_index) + 1,output_dim=100,weights=[embedding_matrix],input_length=100,trainable=False)

In [ ]:
model5 = Sequential()
model5.add(embedding_layer)
model5.add(Conv1D(filters=64,kernel_size=5,strides=1,activation='relu'))
model5.add(MaxPooling1D((3)))
model5.add(LSTM(100))
model5.add(Dense(128,activation= 'relu'))
model5.add(Dense(6,activation='softmax'))

In [ ]:
model5.summary()

In [ ]:
model5.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
history5 = model5.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,epochs=20,batch_size=32)

In [ ]:
plt.plot(history5.history['accuracy'])
plt.plot(history5.history['val_accuracy'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history5.history['loss'])
plt.plot(history5.history['val_loss'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
score = model5.evaluate(X_test,y_test,verbose=0)
print("loss:  ",score[0])
print("accuracy:  ",score[1])

### Improvements

**Try dropouts and see if you can improve the performance of your models.**


In [ ]:
model7 = Sequential()
model7.add(Embedding(vocab_size, embedding_dim))
model7.add(Conv1D(filters=64,kernel_size=5,strides=1,activation='relu'))
model7.add(MaxPooling1D((3)))
model7.add(LSTM(embedding_dim))
model7.add(Dropout(0.15))
model7.add(Dense(238,activation= 'relu'))
model7.add(Dense(6,activation='softmax'))

In [ ]:
model7.summary()

In [ ]:
model7.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
history7 = model7.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,epochs=20,batch_size=32)

In [ ]:
plt.plot(history7.history['accuracy'])
plt.plot(history7.history['val_accuracy'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history7.history['loss'])
plt.plot(history7.history['val_loss'])

plt.title('model loss')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
score = model7.evaluate(X_test,y_test,verbose=0)
print("loss:  ",score[0])
print("accuracy:  ",score[1])

**Split your dataset with 20% testing and observe your performance.**

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_padded,label_seq,train_size=0.8)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
history9 = model5.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,epochs=20,batch_size=32)

In [ ]:
plt.plot(history9.history['accuracy'])
plt.plot(history9.history['val_accuracy'])

plt.title('model accuarcy')
plt.xlabel('accuracy')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history9.history['loss'])
plt.plot(history9.history['val_loss'])

plt.title('model loss')
plt.xlabel('loss')
plt.ylabel('epoch')

plt.legend(['train','validation'])
plt.show()

In [ ]:
score = model9.evaluate(X_test,y_test,verbose=0)
print("loss:  ",score[0])
print("accuracy:  ",score[1])

### Predictions

In [ ]:
txt = ["Australia claimed the crucial wicket of Sri Lankan opener Pathum Nissanka as Pat Cummins struck early-on. Earlier, Australian captain Aaron Finch won the toss and decided to bowl vs Sri Lanka in a Super 12 game in the T20 World Cup 2021 at the Dubai International Stadium in Dubai. Sri Lanka made one change as Maheesh Theekshana was declared fit and came in place of Binura Fernando while Australia went unchanged from their last match. Australia defeated South Africa in their opening game by five wickets and two balls to spare. Sri Lanka, on the other hand, registered a win as well where they went past Bangladesh by five wickets and seven balls remaining.Dasun Shanaka will aim to grab a sensational win today in a tough group which promises to go down to the wire"]

tokenizer = Tokenizer(num_words=5000,oov_token='<oov>')
tokenizer.fit_on_texts(txt)
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=400)

pred = model1.predict(padded)

labels = ['tech','bussiness','politics','sport','entertainment']
print(pred)
print(labels[np.argmax(pred)])

In [ ]:
txt = ["Australia claimed the crucial wicket of Sri Lankan opener Pathum Nissanka as Pat Cummins struck early-on. Earlier, Australian captain Aaron Finch won the toss and decided to bowl vs Sri Lanka in a Super 12 game in the T20 World Cup 2021 at the Dubai International Stadium in Dubai. Sri Lanka made one change as Maheesh Theekshana was declared fit and came in place of Binura Fernando while Australia went unchanged from their last match. Australia defeated South Africa in their opening game by five wickets and two balls to spare. Sri Lanka, on the other hand, registered a win as well where they went past Bangladesh by five wickets and seven balls remaining.Dasun Shanaka will aim to grab a sensational win today in a tough group which promises to go down to the wire"]

tokenizer = Tokenizer(num_words=5000,oov_token='<oov>')
tokenizer.fit_on_texts(txt)
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=400)

pred = model3.predict(padded)

labels = ['tech','bussiness','politics','sport','entertainment']
print(pred)
print(labels[np.argmax(pred)])

In [ ]:
txt = ["Australia claimed the crucial wicket of Sri Lankan opener Pathum Nissanka as Pat Cummins struck early-on. Earlier, Australian captain Aaron Finch won the toss and decided to bowl vs Sri Lanka in a Super 12 game in the T20 World Cup 2021 at the Dubai International Stadium in Dubai. Sri Lanka made one change as Maheesh Theekshana was declared fit and came in place of Binura Fernando while Australia went unchanged from their last match. Australia defeated South Africa in their opening game by five wickets and two balls to spare. Sri Lanka, on the other hand, registered a win as well where they went past Bangladesh by five wickets and seven balls remaining.Dasun Shanaka will aim to grab a sensational win today in a tough group which promises to go down to the wire"]

tokenizer = Tokenizer(num_words=5000,oov_token='<oov>')
tokenizer.fit_on_texts(txt)
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=400)

pred = model7.predict(padded)

labels = ['tech','bussiness','politics','sport','entertainment']
print(pred)
print(labels[np.argmax(pred)])